In [1]:
# -*- coding: utf-8 -*-
"""Hybrid_Sentiment_Analysis.ipynb"""

# Install required libraries
!pip install vaderSentiment transformers scikit-learn pandas numpy

import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from transformers import pipeline

# -------------------------
# 1️⃣ Load Dataset
# -------------------------
df = pd.read_csv("/content/product_reviews_100.csv")  # Make sure you have 'review' & 'sentiment' columns
df.head()

# -------------------------
# 2️⃣ VADER Feature Extraction
# -------------------------
analyzer = SentimentIntensityAnalyzer()

def extract_vader_scores(text):
    scores = analyzer.polarity_scores(text)
    return pd.Series([scores['neg'], scores['neu'], scores['pos'], scores['compound']])

df[['neg','neu','pos','compound']] = df['review'].apply(extract_vader_scores)

# -------------------------
# 3️⃣ TF-IDF Feature Extraction
# -------------------------
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
X_tfidf = tfidf.fit_transform(df['review']).toarray()

# -------------------------
# 4️⃣ Combine VADER + TF-IDF
# -------------------------
X_combined = np.hstack((X_tfidf, df[['neg','neu','pos','compound']].values))

# -------------------------
# 5️⃣ Encode Labels
# -------------------------
le = LabelEncoder()
y_encoded = le.fit_transform(df['sentiment'])

# -------------------------
# 6️⃣ Train-Test Split
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_encoded, test_size=0.2, random_state=42)

# -------------------------
# 7️⃣ Random Forest Training
# -------------------------
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy (RF + VADER + TF-IDF):", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

# -------------------------
# 8️⃣ Transformers Sentiment Pipeline (Optional for Hybrid Boost)
# -------------------------
transformer_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def predict_hybrid_sentiment(text):
    # VADER features
    scores = analyzer.polarity_scores(text)
    vader_features = [scores['neg'], scores['neu'], scores['pos'], scores['compound']]

    # TF-IDF vector
    tfidf_features = tfidf.transform([text]).toarray()[0]

    # Combine
    combined_features = np.hstack((tfidf_features, vader_features)).reshape(1,-1)

    # RF Prediction
    rf_pred = clf.predict(combined_features)[0]

    # Transformer Prediction
    transformer_pred = transformer_pipeline(text)[0]['label']

    # Map Transformer labels to numeric if needed
    transformer_map = {"POSITIVE": 1, "NEGATIVE": 0}  # Adjust based on your dataset
    transformer_pred_num = transformer_map.get(transformer_pred, rf_pred)

    # Hybrid Decision: If both agree, use that, else trust transformer
    final_pred_num = rf_pred if rf_pred == transformer_pred_num else transformer_pred_num

    return le.inverse_transform([final_pred_num])[0]

# -------------------------
# 9️⃣ Test Hybrid Model
# -------------------------
sample_reviews = [
    "I love this phone 😍🔥",
    "This app is trash 😡💩",
    "Hmm 🤔 not sure about this one...",
    "That concert was lit 🔥🔥🔥"
]

for review in sample_reviews:
    print(f"{review} => {predict_hybrid_sentiment(review)}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.3 MB/s eta 0:00:00
Accuracy (RF + VADER + TF-IDF): 0.95

Classification Report:
               precision    recall  f1-score   support

    Negative       0.88      1.00      0.93         7
     Neutral       1.00      1.00      1.00         5
    Positive       1.00      0.88      0.93         8

    accuracy                           0.95        20
   macro avg       0.96      0.96      0.96        20
weighted avg       0.96      0.95      0.95        20



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


I love this phone 😍🔥 => Neutral
This app is trash 😡💩 => Negative
Hmm 🤔 not sure about this one... => Negative
That concert was lit 🔥🔥🔥 => Negative


In [ ]:
from google.colab import drive
drive.mount('/content/drive')